# Quarto Projeto: Analisador Semântico

Depois que a árvore de sintaxe abstrata (do inglês, *Abstract Syntax Tree* - AST) é construída, uma análise adicional pode ser feita avaliando os atributos dos nós da árvore para reunir as informações semânticas necessárias do código-fonte que não são facilmente detectadas durante a análise sintática. Isso geralmente inclui a verificação de tipos, a construção da tabela de símbolos para garantir que uma variável seja declarada antes do uso e a decoração da AST para prepará-la para a próxima fase de compilação.

## Sistema de Tipos

Primeiro, você precisará definir objetos que representem os diferentes tipos de dados básicos e armazenar informações sobre suas características.

Vamos definir classes que representam tipos. Existe uma classe geral usada para representar todos os tipos. Cada tipo básico é então uma única instância da classe de tipo.

```python
class uChuckType:
      pass

int_type = uChuckType("int", ...)
float_type = uChuckType("float", ...)
```

O conteúdo da classe de tipo depende inteiramente de você. No entanto, você precisará codificar minimamente algumas informações sobre quais operadores são suportados (+, -, *, etc.) e valores padrão.

Depois de definir os tipos básicos, você precisará certificar-se de que eles sejam registrados com qualquer tabela de símbolos ou código que verifique os nomes dos tipos.

In [2]:
class Type:
    """
    Class that represents a type in the language. Basic
    Types are declared as singleton instances of this type.
    """

    def __init__(self, name):
        self.typename = name

    def __repr__(self):
        return self.typename

class UChuckType(Type):

    def __init__(self, name, binary_ops=set(), unary_ops=set(), rel_ops=set()):
        super().__init__(name)
        self.binary_ops = binary_ops
        self.unary_ops = unary_ops
        self.rel_ops = rel_ops

# Create specific instances of basic types. You will need to add
# appropriate arguments depending on your definition of Type
IntType = UChuckType(
    "int",
    unary_ops  = {"-", "+", "!"},
    binary_ops = {"+", "-", "*", "/", "%"},
    rel_ops    = {"==", "!=", "<", ">", "<=", ">=", "&&", "||"},
)

FloatType = UChuckType(
    "float",
    unary_ops  = {"-", "+"},
    binary_ops = {"+", "-", "*", "/", "%"},
    rel_ops    = {"==", "!=", "<", ">", "<=", ">="},
)

StringType = UChuckType(
    "string",
    binary_ops = {"+"},
    rel_ops    = {"==", "!="},
)

Em seu código de verificação de tipos, você precisará fazer referência aos objetos de tipo acima. Pense em como você vai querer acessá-los.

## Tabela de Símbolos

Você precisará definir uma tabela de símbolos que acompanhe os identificadores declarados anteriormente. A tabela de símbolos será consultada sempre que o compilador precisar consultar informações sobre declarações de variáveis e constantes.

In [4]:
class SymbolTable(dict):
    """ Class representing a symbol table. It should provide functionality
        for adding and looking up nodes associated with identifiers.
    """
    def __init__(self, parent=None):
        super().__init__()

    def add(self, name, value):
        self[name] = value

    def lookup(self, name):
        return self.get(name, None)

Observe que a análise semântica terá que gerenciar a tabela de símbolos para lidar com os múltiplos escopos do programa.

## Visitando a AST

O padrão de visitante é frequentemente usado no compilador para percorrer estruturas de dados que representam os programas, seja árvore de sintaxe ou qualquer outra representação intermediária. Para isso, fornecemos a seguinte classe `NodeVisitor` para permitir que você visite a AST. Essa classe foi modelada com base nas instalações de visitação AST do próprio Python (o módulo AST do Python 3).

In [5]:
class NodeVisitor:
    """ A base NodeVisitor class for visiting ucyan_ast nodes.
        Subclass it and define your own visit_XXX methods, where
        XXX is the class name you want to visit with these
        methods.
    """

    _method_cache = None

    def visit(self, node):
        """ Visit a node.
        """

        if self._method_cache is None:
            self._method_cache = {}

        visitor = self._method_cache.get(node.__class__.__name__, None)
        if visitor is None:
            method = 'visit_' + node.__class__.__name__
            visitor = getattr(self, method, self.generic_visit)
            self._method_cache[node.__class__.__name__] = visitor

        return visitor(node)

    def generic_visit(self, node):
        """ Called if no explicit visitor function exists for a
            node. Implements preorder visiting of the node.
        """
        for _, child in node.children():
            self.visit(child)

Por exemplo, um pequeno visitante para literais pode ser implementado desta forma:

In [6]:
class LiteralVisitor(NodeVisitor):
    def __init__(self):
        self.values = []

    def visit_Literal(self, node):
        self.values.append(node.value)

Esse visitante criaria uma lista de valores de todos os nós literais encontrados abaixo de um determinado nó. Para usá-lo, basta instanciar o visitante e chamar seu método `visit` no nó de sua escolha:

```python
lv = LiteralVisitor()
lv.visit(node)
```

Observe que:

- O método `generic_visit()` será chamado para nós AST para os quais nenhum método `visit_XXX` foi definido.
- Os filhos de nós para os quais um método `visit_XXX` foi definido não serão visitados - se você precisar disso, chame `generic_visit()` no nó.
- O método `generic_visit()` pode ser implementado de forma mais eficiente definindo os nós AST como objeto iterável usando o método `__iter__()` do Python. Sinta-se à vontade para otimizar.

## Erros Padronizados

Para verificar erros semânticos e imprimir sua descrição, deve-se usar o seguinte método:

```python
    def _assert_semantic(self, condition, msg_code, coord, name="", ltype="", rtype=""):
        """Check condition, if false print selected error message and exit"""
        error_msgs = {
             1: f"'{name}' is not defined",
             2: f"Cannot assign type '{rtype}' to type '{ltype}'",
             3: f"Binary operator '{name}' does not have matching LHS/RHS types",
             4: f"Binary operator '{name}' is not supported by type '{ltype}'",
             5: "Break/Continue statement must be inside a loop",
             6: f"The condition expression must be of type 'int', not type '{ltype}'",
             7: "Expression is not of basic type",
             8: f"Right-side operand is not a variable",
             9: f"Name '{name}' is already defined in this scope",
            10: f"Unary operator '{name}' is not supported by type '{ltype}'",
        }
        if not condition:
            msg = error_msgs.get(msg_code)
            print("SemanticError: %s %s" % (msg, coord), file=sys.stdout)
            sys.exit(1)
```

Conforme mostrado no código da função, cada mensagem está associada a um código de erro específico. A mensagem de erro é impressa de acordo com as coordenadas fornecidas e os argumentos opcionais. O método `_assert_semantic` é usado para padronizar a saída da análise semântica e deve ser usado para passar nos testes automáticos. Alguns exemplos que mostram como usar o método são fornecidos na próxima seção.

## Implementando o Análisador Semântico

Você precisará escrever um código que percorra a AST, decore-a com informações adicionais e imponha um conjunto de regras semânticas, conforme explicado pelas diretrizes abaixo. Para percorrer a AST, use a classe `NodeVisitor`. Uma implementação inicial do analisador semântico é fornecida no código abaixo.

```python
class Visitor(NodeVisitor):
    '''
    Program visitor class. This class uses the visitor pattern. You need to define methods
    of the form visit_NodeName() for each kind of AST node that you want to process.
    '''
    def __init__(self):
        # Keep a reference to current symbol table
        self.symtab = None

        # Add built-in type names (int, float, char, bool)
        self.typemap = {
            "int": IntType,
            "float": FloatType,
            "string": StringType,
        }
        
    def visit_ChuckOp(self, node):
        # Visit the expression (on the left)
        self.visit(node.expression)
        expr_type = node.expression.attrs['uchuck_type']
        # Visit the location (on the right)
        self.visit(node.location)
        loc_type = node.location.attrs['uchuck_type']
        # Make sure the location is a variable, otherwise, return an error (code 8)
        loc = node.location
        self._assert_semantic('defn' in loc.attrs, 8, coord=loc.coord)        
        # Check that the assignment is allowed otherwise return a type error (code 2)
        self._assert_semantic(loc_type == expr_type, 2, node.coord, ltype=loc_type, rtype=expr_type)
        # Assign the type of the location to current node
        node.attrs['uchuck_type'] = loc_type

    def visit_BinaryOp(self, node):
        # Visit the left and right expression
        self.visit(node.left)
        ltype = node.left.attrs['uchuck_type']
        self.visit(node.right)
        rtype = node.right.attrs['uchuck_type']
        # TODO:
        # - Make sure left and right operands have the same type
        # - Make sure the operation is supported
        # - Assign the result type to current node

    def visit_Program(self, node):
        # Create a symbol table and assign it to current node
        node.attrs['symtab'] = SymbolTable()
        # Set the reference to current symbol table to the new symbol table
        self.symtab = node.attrs['symtab']
        # Create an empty list to bind loop nodes to nested break/continue statements and insert it into the symbol table
        self.symtab.add('loops', [])
        # Visit all the statements
        for stmt in node.stmts:
            self.visit(stmt)
```

**IMPORTANTE:** A AST que você criou anteriormente contém apenas informações (como tipos) em nós específicos. Além de encontrar os possíveis erros remanescentes do programa, a análise semântica deve ser usada para descobrir informações adicionais (como o tipo de todas as expressões) que serão úteis para a geração de código, a próxima fase de compilação. Este processo é geralmente chamado de "decorar a AST".

## Diretrizes

Além disso, fornecemos um conjunto de diretrizes que podem ser usadas para implementar cada função da análise semântica (verificação de tipo, verificação de definição, etc). Por favor, leia-as com atenção.

### Programa

1. Programa (`visit_Program`)

- Crie uma tabela de símbolos e atribua-a ao nó atual (`attrs['symtab']`)
- Ajuste a referência à tabela de símbolos atual para a nova tabela de símbolos
- Crie uma lista vazia para vincular laços de repetição (*loops*) a comandos *break*/*continue* aninhados e insira-a na tabela de símbolos
- Visite todos os comandos (`stmts`)

### Declarações / Tipo

1. Definição de variável (`visit_VarDecl`)

- Obtenha o nome (`name`) da variável e verifique se ela não está definida, caso contrário, retorne um erro (código 9)
- Visite o tipo (`dtype`) e obtenha seu valor
- Insira o nome (`name`) da variável na tabela de símbolos
- Atribua o tipo ao nó atual (`attrs['uchuck_type']`)

2. Tipo (`visit_Type`)

- Certifique-se de que o nome (`name`) do tipo seja de um tipo básico ou retorne um erro (código 1)
- Obtenha o objeto `uChuckType` básico correspondente ao tipo (`name`) e atribua-o ao nó atual (`attrs['uchuck_type']`)

### Comandos

1. If (`visit_IfStatement`)

- Primeiro, visite a condição (`test`)
- Em seguida, verifique se a expressão condicional é do tipo inteiro ou retorne um erro de tipo (código 6)
- Por fim, visite as declarações relacionadas ao "então" (`consequence`) e ao "senão" (`alternative`), se houver

```
if( 3.0 ) { } // SemanticError: The condition expression must be of type 'int', not type 'float'
```

2. While (`visit_WhileStatement`)

- Procure a lista de laços de repetição (*loops*) na tabela de símbolos
- Adicione o nó de laço de repetição atual no final da lista
- Visite a condição (`test`)
- Verifique se a expressão condicional é do tipo inteiro ou retorne um erro de tipo (código 6)
- Visite todos os comandos relacionados ao corpo do laço (`body`)
- Retire o nó do laço de repetição atual da lista

3. Break (`visit_BreakStatement`)

- Primeiro, procure a lista de laços de repetição (*loops*) na tabela de símbolos
- Em seguida, verifique se o comando *break* está dentro de um laço de repetição, caso contrário, deve retornar um erro (código 5)
- Por fim, vincule-o ao nó do laço de repetição atual (`attrs['loop']`)

4. Continue (`visit_ContinueStatement`)

- Primeiro, procure a lista de laços de repetição (*loops*) na tabela de símbolos
- Em seguida, verifique se o comando *continue* está dentro de um laço de repetição, caso contrário, deve retornar um erro (código 5)
- Por fim, vincule-o ao nó do laço de repetição atual (`attrs['loop']`)

5. Expressão como comando (`visit_ExpressionAsStatement`)

- Se não estiver vazio, visite a expressão (`expression`)

6. Bloco (`visit_StmtList`)

- Se não estiver vazio, visite cada comando (`stmts`)

### Expressões

1. Literal (`visit_Literal`)

- Obtenha o objeto `uChuckType` correspondente ao seu tipo (`type`) e atribua-o ao nó atual (`attrs['uchuck_type']`)

2. Local (`visit_Location`)

- Procure a declaração do local (`name`) na tabela de símbolos
- Certifique-se de que ele está definido, caso contrário retorne um erro (código 1)
- Vincule o nó de declaração do local ao nó atual (`attrs['defn']`)
- Atribua o tipo do nó de declaração do local ao nó atual (`attrs['uchuck_type']`)

3. Operação binária (`visit_BinaryOp`)

- Comece visitando cada operando da operação (`left`, `right`)
- Verifique se ambos os operandos possuem o mesmo tipo ou retorne um erro de tipo (código 3)
- Verifique se o operador (`op`) da operação binária é compatível com o tipo dos operandos, tentativas de usar operadores não suportados devem resultar em erro (código 4)
- Verifique o tipo do resultado:
    - Operações binárias usando operador aritmético produzem um resultado do mesmo tipo que os operandos
    - Operações binárias usando operador relacional produzem um resultado do tipo inteiro (`IntType`)
- Atribua o tipo do resultado ao nó atual (`attrs['uchuck_type']`)

Aqui está a lista completa de operadores suportados por cada tipo:

```python
    # int:
        binary_ops = {"+", "-", "*", "/", "%"}
        rel_ops    = {"==", "!=", "<", ">", "<=", ">="}

    # float:
        binary_ops = {"+", "-", "*", "/", "%"}
        rel_ops    = {"==", "!=", "<", ">", "<=", ">="}

    # string:
        bin_ops    = {"+"}
        rel_ops    = {"==", "!="}
```

Por exemplo:

```
        2 => int a;
        "3" => string b;

        a + 3 => int c;    // OK
        a + b => int d;    // Error.  int + string
        b + 4 => int e;    // Error.  string + int
```

4. Operação unária (`visit_UnaryOp`)

- Comece visitando o operando da operação (`operand`)
- Verifique se o operador (`op`) da operação é compatível com o tipo do operando, tentativas de usar operadores não suportados devem resultar em erro (código 10)
- Defina o tipo do nó atual que representa a operação unária com o mesmo tipo do operando (`attrs['uchuck_type']`)

Aqui está a lista completa de operadores suportados por cada tipo:

```python
    # int:
        unary_ops  = {"-", "+", "!"}

    # float:
        unary_ops  = {"-", "+"}
```

5. Atribuição (`visit_ChuckOp`)

- Visite o valor (`expression`) (à esquerda)
- Visite o local (`location`) (à direita)
- Certifique-se de que o local seja uma variável, caso contrário, retorne um erro (código 8)
- Verifique se a atribuição é permitida, caso contrário, retorne um erro de tipo (código 2): os lados esquerdo e direito de uma operação de atribuição devem ser declarados como do mesmo tipo (`attrs['uchuck_type']`)
- Atribua o tipo do local ao nó atual (`attrs['uchuck_type']`)

Veja o exemplo abaixo:

```
"3" => string c;
c => int j;             // SemanticError: Cannot assign type 'string' to type 'int'
```

6. Impressão (`visit_PrintStatement`)

- Visite a expressão (`expression`)
- Certifique-se de que a expressão seja de um tipo básico ou retorne um erro de tipo (código 7)
- Atribua o tipo da expressão ao nó atual (`attrs['uchuck_type']`)

7. Lista de expressões (`visit_ExprList`)

- Visite cada expressão (`exprs`)
- Atribua o tipo da última expressão ao nó atual (`attrs['uchuck_type']`)

## Esboço do Analisador Semântico

In [7]:
!pip install sly
from sly import Lexer, Parser

Copie o código da classe `UChuckLexer` que você escreveu no [primeiro projeto](https://colab.research.google.com/drive/1FXsbf3mp9rf-Cce-c9qk_8S43ppcPd4k?usp=sharing) e cole na célula abaixo.

In [8]:
class UChuckLexer(Lexer):
    """A lexer for the uChuck language."""

    def __init__(self, error_func):
        """Create a new Lexer.
        An error function. Will be called with an error
        message, line and column as arguments, in case of
        an error during lexing.
        """
        self.error_func = error_func

    # Reserved keywords
    keywords = {
        'int': "INT",
        'float': "FLOAT",
        'if': "IF",
        'else': "ELSE",
        'while': "WHILE",
        'break': "BREAK",
        'continue': "CONTINUE",
        'true': "TRUE",
        'false': "FALSE",
    }

    # All the tokens recognized by the lexer
    tokens = tuple(keywords.values()) + (
        # Identifiers
        "ID",
        # Constants
        "FLOAT_VAL",
        "INT_VAL",
        "STRING_LIT",
        # Operators
        "PLUS",
        "MINUS",
        "TIMES",
        "DIVIDE",
        "PERCENT",
        "LE",
        "LT",
        "GE",
        "GT",
        "EQ",
        "NEQ",
        "AND",
        "OR",
        "EXCLAMATION",
        # Assignment
        "CHUCK",
        # Delimeters
        "LPAREN",
        "RPAREN",  # ( )
        "LBRACE",
        "RBRACE",  # { }
        "L_HACK",
        "R_HACK",  # <<< >>>
        "COMMA",
        "SEMI",  # , ;
    )

    # String containing ignored characters (between tokens)
    ignore = " \t"

    # Other ignored patterns
    ignore_newline = r'\n+'
    ignore_comment = r'/\*(.|\n)*?\*/|//.*'

    # Regular expression rules for tokens
    ID = r'[a-zA-Z_][a-zA-Z0-9_]*'
    FLOAT_VAL = r'\d+\.\d+|\d+\.\d*|\.\d+'
    INT_VAL = r'\d+'
    STRING_LIT = r'"([^"\\]|\\.)*"'
    
    unterm_string = r'"(\\["\\]|[^"\\])*'
    unterm_comment = r'/\*.*'

    L_HACK = r'<<<'
    R_HACK = r'>>>'

    LE = r'<='
    GE = r'>='
    EQ = r'=='
    NEQ = r'!='
    AND = r'&&'
    OR = r'\|\|'
    CHUCK = r'=>'

    LT = r'<'
    GT = r'>'
    PLUS = r'\+'
    MINUS = r'-'
    TIMES = r'\*'
    DIVIDE = r'/'
    PERCENT = r'%'
    EXCLAMATION = r'!'

    LPAREN = r'\('
    RPAREN = r'\)'
    LBRACE = r'\{'
    RBRACE = r'\}'
    COMMA = r','
    SEMI = r';'

    # Special cases
    def ID(self, t):
      t.type = self.keywords.get(t.value, "ID")
      return t

    # Define a rule so we can track line numbers
    def ignore_newline(self, t):
        self.lineno += len(t.value)

    def ignore_comment(self, t):
        self.lineno += t.value.count("\n")

    def find_column(self, token):
        """Find the column of the token in its line."""
        last_cr = self.text.rfind('\n', 0, token.index)
        return token.index - last_cr

    # Internal auxiliary methods
    def _error(self, msg, token):
        location = self._make_location(token)
        self.error_func(msg, location[0], location[1])
        self.index += 1

    def _make_location(self, token):
        return token.lineno, self.find_column(token)

    def unterm_comment(self, t):
        msg = "Unterminated comment"
        self._error(msg, t)

    def unterm_string(self, t):
        msg = "Unterminated string literal"
        self._error(msg, t)

    def error(self, t):
        msg = "Illegal character %s" % repr(t.value[0])
        self._error(msg, t)

    # Scanner (used only for test)
    def scan(self, text):
        output = ""
        for tok in self.tokenize(text):
            print(tok)
            output += str(tok) + "\n"
        return output

Copie o código das classes de nós AST que você escreveu no [terceiro projeto](https://colab.research.google.com/drive/1fRu6iseV-iWDrMtP7nR3yjame1RGEpnN?usp=sharing) e cole na célula abaixo.

In [9]:
import sys

def represent_node(obj, indent):
    def _repr(obj, indent, printed_set):
        """
        Get the representation of an object, with dedicated pprint-like format for lists.
        """
        if isinstance(obj, list):
            indent += 1
            sep = ",\n" + (" " * indent)
            final_sep = ",\n" + (" " * (indent - 1))
            return (
                "["
                + (sep.join((_repr(e, indent, printed_set) for e in obj)))
                + final_sep
                + "]"
            )
        elif isinstance(obj, Node):
            if obj in printed_set:
                return ""
            else:
                printed_set.add(obj)
            result = obj.__class__.__name__ + "("
            indent += len(obj.__class__.__name__) + 1
            attrs = []
            for name in obj.__slots__:
                if name == "bind":
                    continue
                value = getattr(obj, name)
                value_str = _repr(value, indent + len(name) + 1, printed_set)
                attrs.append(name + "=" + value_str)
            sep = ",\n" + (" " * indent)
            final_sep = ",\n" + (" " * (indent - 1))
            result += sep.join(attrs)
            result += ")"
            return result
        elif isinstance(obj, str):
            return obj
        else:
            return ""

    # avoid infinite recursion with printed_set
    printed_set = set()
    return _repr(obj, indent, printed_set)

class Coord:
    """Coordinates of a syntactic element. Consists of:
    - Line number
    - (optional) column number, for the Lexer
    """

    __slots__ = ("line", "column")

    def __init__(self, line, column=None):
        self.line = line
        self.column = column

    def __str__(self):
        if self.line and self.column is not None:
            coord_str = "@ %s:%s" % (self.line, self.column)
        elif self.line:
            coord_str = "@ %s" % (self.line)
        else:
            coord_str = ""
        return coord_str

class Node:
    """Abstract base class for AST nodes."""

    __slots__ = ("coord", "attrs",)

    def __init__(self, coord=None):
        self.coord = coord
        self.attrs = {}

    def children(self):
        """A sequence of all children that are Nodes"""
        pass

    attr_names = ()

    def __repr__(self):
        """Generates a python representation of the current node"""
        return represent_node(self, 0)

    def show(
        self,
        buf=sys.stdout,
        offset=0,
        attrnames=False,
        nodenames=False,
        showcoord=False,
        _my_node_name=None,
    ):
        """Pretty print the Node and all its attributes and children (recursively) to a buffer.
        buf:
            Open IO buffer into which the Node is printed.
        offset:
            Initial offset (amount of leading spaces)
        attrnames:
            True if you want to see the attribute names in name=value pairs. False to only see the values.
        nodenames:
            True if you want to see the actual node names within their parents.
        showcoord:
            Do you want the coordinates of each Node to be displayed.
        """
        lead = " " * offset
        if nodenames and _my_node_name is not None:
            buf.write(lead + self.__class__.__name__ + " <" + _my_node_name + ">: ")
            inner_offset = len(self.__class__.__name__ + " <" + _my_node_name + ">: ")
        else:
            buf.write(lead + self.__class__.__name__ + ":")
            inner_offset = len(self.__class__.__name__ + ":")

        if self.attr_names:
            if attrnames:
                nvlist = [
                    (n, represent_node(getattr(self, n), offset+inner_offset+1+len(n)+1))
                    for n in self.attr_names
                    if getattr(self, n) is not None
                ]
                attrstr = ", ".join("%s=%s" % nv for nv in nvlist)
            else:
                vlist = [getattr(self, n) for n in self.attr_names]
                attrstr = ", ".join(
                    represent_node(v, offset + inner_offset + 1) for v in vlist
                )
            buf.write(" " + attrstr)

        if showcoord:
            if self.coord and self.coord.line != 0:
                buf.write(" %s" % self.coord)
        buf.write("\n")

        for (child_name, child) in self.children():
            child.show(buf, offset + 4, attrnames, nodenames, showcoord, child_name)

class NodeVisitor:
    """ A base NodeVisitor class for visiting uchuck_ast nodes.
        Subclass it and define your own visit_XXX methods, where
        XXX is the class name you want to visit with these
        methods.
    """

    _method_cache = None

    def visit(self, node):
        """ Visit a node.
        """

        if self._method_cache is None:
            self._method_cache = {}

        visitor = self._method_cache.get(node.__class__.__name__, None)
        if visitor is None:
            method = 'visit_' + node.__class__.__name__
            visitor = getattr(self, method, self.generic_visit)
            self._method_cache[node.__class__.__name__] = visitor

        return visitor(node)

    def generic_visit(self, node):
        """ Called if no explicit visitor function exists for a
            node. Implements preorder visiting of the node.
        """
        for _, child in node.children():
            self.visit(child)

class BinaryOp(Node):

    __slots__ = ("op", "left", "right",)

    def __init__(self, op, left, right, coord=None):
        super().__init__(coord)
        self.op = op
        self.left = left
        self.right = right

    def children(self):
        nodelist = []
        if self.left is not None:
            nodelist.append(('left', self.left))
        if self.right is not None:
            nodelist.append(('right', self.right))
        return tuple(nodelist)

    attr_names = ("op",)

class BreakStatement(Node):
    __slots__ = ()

    def __init__(self, coord=None):
        super().__init__(coord)

    def children(self):
        return ()

    attr_names = ()

class ChuckOp(Node):
    __slots__ = ("left", "right")

    def __init__(self, left, right, coord=None):
        super().__init__(coord)
        self.left = left
        self.right = right

    def children(self):
        return (('right', self.right), ('left', self.left))

    attr_names = ()

class ContinueStatement(Node):
    __slots__ = ()

    def __init__(self, coord=None):
        super().__init__(coord)

    def children(self):
        return ()

    attr_names = ()

class ExpressionAsStatement(Node):
    __slots__ = ("expression",)

    def __init__(self, expression, coord=None):
        super().__init__(coord)
        self.expression = expression

    def children(self):
        if self.expression is not None:
            return (('expression', self.expression),)
        return ()

    attr_names = ()

class ExprList(Node):
    __slots__ = ("exprs",)

    def __init__(self, exprs, coord=None):
        super().__init__(coord)
        self.exprs = exprs

    def children(self):
        nodelist = []
        for i, child in enumerate(self.exprs or []):
            nodelist.append(('exprs[%d]' % i, child))
        return tuple(nodelist)

    attr_names = ()

class ID(Node):
    __slots__ = ("name",)

    def __init__(self, name, coord=None):
        super().__init__(coord)
        self.name = name

    def children(self):
        return ()

    attr_names = ("name",)

class IfStatement(Node):
    __slots__ = ("test", "consequence", "alternative")

    def __init__(self, test, consequence, alternative, coord=None):
        super().__init__(coord)
        self.test = test
        self.consequence = consequence
        self.alternative = alternative

    def children(self):
        nodelist = []
        if self.test is not None:
            nodelist.append(('test', self.test))
        if self.consequence is not None:
            nodelist.append(('consequence', self.consequence))
        if self.alternative is not None:
            nodelist.append(('alternative', self.alternative))
        return tuple(nodelist)

    attr_names = ()

class Literal(Node):

    __slots__ = ("type", "value",)

    def __init__(self, type, value, coord=None):
        super().__init__(coord)
        self.type = type
        self.value = value

    def children(self):
        return ()

    attr_names = ("type", "value",)

class Location(Node):
    __slots__ = ("name",)

    def __init__(self, name, coord=None):
        super().__init__(coord)
        self.name = name

    def children(self):
        return ()

    attr_names = ("name",)

class PrintStatement(Node):
    __slots__ = ("expression",)

    def __init__(self, expression, coord=None):
        super().__init__(coord)
        self.expression = expression

    def children(self):
        if self.expression is not None:
            return (('expression', self.expression),)
        return ()

    attr_names = ()

class Program(Node):

    __slots__ = ("stmts",)

    def __init__(self, stmts, coord=None):
        super().__init__(coord)
        self.stmts = stmts

    def children(self):
        nodelist = []
        for i, child in enumerate(self.stmts or []):
            nodelist.append(('stmts[%d]' % i, child))
        return tuple(nodelist)

class StmtList(Node):
    __slots__ = ("stmts",)

    def __init__(self, stmts, coord=None):
        super().__init__(coord)
        self.stmts = stmts

    def children(self):
        nodelist = []
        for i, child in enumerate(self.stmts or []):
            nodelist.append(('stmts[%d]' % i, child))
        return tuple(nodelist)

    attr_names = ()

class Type(Node):
    __slots__ = ("name",)

    def __init__(self, name, coord=None):
        super().__init__(coord)
        self.name = name

    def children(self):
        return ()

    attr_names = ("name",)

class UnaryOp(Node):
    __slots__ = ("op", "operand")

    def __init__(self, op, operand, coord=None):
        super().__init__(coord)
        self.op = op
        self.operand = operand

    def children(self):
        nodelist = []
        if self.operand is not None:
            nodelist.append(('operand', self.operand))
        return tuple(nodelist)

    attr_names = ("op",)

class VarDecl(Node):
    __slots__ = ("dtype", "name")

    def __init__(self, dtype, name, coord=None):
        super().__init__(coord)
        self.dtype = dtype
        self.name = name

    def children(self):
        nodelist = []
        if self.dtype is not None:
            nodelist.append(('dtype', self.dtype))
        return tuple(nodelist)

    attr_names = ("name",)

class WhileStatement(Node):
    __slots__ = ("test", "body")

    def __init__(self, test, body, coord=None):
        super().__init__(coord)
        self.test = test
        self.body = body

    def children(self):
        nodelist = []
        if self.test is not None:
            nodelist.append(('test', self.test))
        if self.body is not None:
            nodelist.append(('body', self.body))
        return tuple(nodelist)

    attr_names = ()

Copie o código da classe `UChuckParser` que você escreveu no [terceiro projeto](https://colab.research.google.com/drive/1fRu6iseV-iWDrMtP7nR3yjame1RGEpnN?usp=sharing) e cole na célula abaixo.

In [10]:
class UChuckParser(Parser):
    """A parser for the uChuck language."""

    # Get the token list from the lexer (required)
    tokens = UChuckLexer.tokens

    precedence = (
        ('left', 'COMMA'),
        ('left', 'CHUCK'),
        ('left', 'OR'),
        ('left', 'AND'),
        ('left', 'EQ', 'NEQ'),
        ('left', 'LT', 'LE', 'GT', 'GE'),
        ('left', 'PLUS', 'MINUS'),
        ('left', 'TIMES', 'DIVIDE', 'PERCENT'),
        ('right', 'EXCLAMATION'),
    )

    def __init__(self, error_func=lambda msg, x, y: print("Lexical error: %s at %d:%d" % (msg, x, y), file=sys.stdout)):
        self.lexer = UChuckLexer(error_func)

    def parse(self, text, lineno=1, index=0):
        return super().parse(self.lexer.tokenize(text, lineno, index))

    def _token_coord(self, p):
        return Coord(p.lineno, self.lexer.find_column(p))

    def error(self, p):
        if p:
            if hasattr(p, 'lineno'):
                print("Error at line %d near the symbol %s " % (p.lineno, p.value))
            else:
                print("Error near the symbol %s" % p.value)
        else:
            print("Error at the end of input")

    # <program> ::= <statement_list> EOF
    @_('statement_list')
    def program(self, p):
        return Program(p.statement_list)

    # <statement_list> ::= { <statement> }+
    @_('statement statement_list')
    def statement_list(self, p):
        return [p.statement] + p.statement_list

    @_('statement')
    def statement_list(self, p):
        return [p.statement]

    # <statement> ::= <expression_statement>
    #               | <loop_statement>
    #               | <selection_statement>
    #               | <jump_statement>
    #               | <code_segment>
    @_('expression_statement',
       'loop_statement',
       'selection_statement',
       'jump_statement',
       'code_segment')
    def statement(self, p):
        return p[0]

    # <jump_statement> ::= "break" ";"
    #                    | "continue" ";"
    @_('BREAK SEMI')
    def jump_statement(self, p):
        return BreakStatement(coord=self._token_coord(p))

    @_('CONTINUE SEMI')
    def jump_statement(self, p):
        return ContinueStatement(coord=self._token_coord(p))

    # <selection_statement> ::= "if" "(" <expression> ")" <statement> { "else" <statement> }?
    @_('IF LPAREN expression RPAREN statement ELSE statement')
    def selection_statement(self, p):
        return IfStatement(test=p.expression, consequence=p.statement0, alternative=p.statement1, coord=self._token_coord(p))

    @_('IF LPAREN expression RPAREN statement')
    def selection_statement(self, p):
        return IfStatement(test=p.expression, consequence=p.statement, alternative=None, coord=self._token_coord(p))

    # <loop_statement> ::= "while" "(" <expression> ")" <statement>
    @_('WHILE LPAREN expression RPAREN statement')
    def loop_statement(self, p):
        return WhileStatement(test=p.expression, body=p.statement, coord=self._token_coord(p))

    # <code_segment> ::= "{" { <statement_list> }? "}"
    @_('LBRACE statement_list RBRACE')
    def code_segment(self, p):
        return StmtList(stmts=p.statement_list, coord=self._token_coord(p))

    @_('LBRACE RBRACE')
    def code_segment(self, p):
        return StmtList(stmts=[], coord=self._token_coord(p))

    # <expression_statement> ::= { <expression> }? ";"
    @_('expression SEMI')
    def expression_statement(self, p):
        return ExpressionAsStatement(expression=p.expression, coord=None)

    @_('SEMI')
    def expression_statement(self, p):
        return ExpressionAsStatement(None, coord=None)

    # <expression> ::= <chuck_expression> { "," <chuck_expression> }*
    @_('expression COMMA expression')
    def expression(self, p):
        if isinstance(p.expression0, ExprList):
            expressions = p.expression0.exprs + [p.expression1]
            coord = p.expression0.coord
        else:
            expressions = [p.expression0, p.expression1]
            coord = p.expression0.coord
        
        return ExprList(exprs=expressions, coord=coord)

    @_('chuck_expression')
    def expression(self, p):
        return p.chuck_expression

    # <chuck_expression> ::= { <chuck_expression> "=>" }? <decl_expression>
    @_('chuck_expression CHUCK decl_expression')
    def chuck_expression(self, p):
        return ChuckOp(left=p.chuck_expression, right=p.decl_expression, coord=self._token_coord(p))
    
    @_('decl_expression')
    def chuck_expression(self, p):
        return p.decl_expression

    # <decl_expression> ::= <binary_expression>
    #                     | <type_decl> <identifier>
    @_('binary_expression')
    def decl_expression(self, p):
        return p.binary_expression

    @_('type_decl identifier')
    def decl_expression(self, p):
        return VarDecl(dtype=p.type_decl, name=ID(name=p.identifier.name, coord=p.identifier.coord), coord=None)

    # <type_decl> ::= "int"
    #               | "float"
    #               | <identifier>
    @_('INT')
    def type_decl(self, p):
        return Type(name='int', coord=self._token_coord(p))

    @_('FLOAT')
    def type_decl(self, p):
        return Type(name='float', coord=self._token_coord(p))

    @_('identifier')
    def type_decl(self, p):
        return Type(name=p.identifier.name, coord=p.identifier.coord)

    # <binary_expression> ::= <unary_expression>
    #                       | <binary_expression> "+"  <binary_expression>
    #                       | <binary_expression> "-"  <binary_expression>
    #                       | <binary_expression> "*"  <binary_expression>
    #                       | <binary_expression> "/"  <binary_expression>
    #                       | <binary_expression> "%"  <binary_expression>
    #                       | <binary_expression> "<=" <binary_expression>
    #                       | <binary_expression> "<"  <binary_expression>
    #                       | <binary_expression> ">=" <binary_expression>
    #                       | <binary_expression> ">"  <binary_expression>
    #                       | <binary_expression> "==" <binary_expression>
    #                       | <binary_expression> "!=" <binary_expression>
    #                       | <binary_expression> "&&" <binary_expression>
    #                       | <binary_expression> "||" <binary_expression>
    @_('binary_expression PLUS binary_expression',
       'binary_expression MINUS binary_expression',
       'binary_expression TIMES binary_expression',
       'binary_expression DIVIDE binary_expression',
       'binary_expression PERCENT binary_expression',
       'binary_expression LE binary_expression',
       'binary_expression LT binary_expression',
       'binary_expression GE binary_expression',
       'binary_expression GT binary_expression',
       'binary_expression EQ binary_expression',
       'binary_expression NEQ binary_expression',
       'binary_expression AND binary_expression',
       'binary_expression OR binary_expression',
       )
    def binary_expression(self, p):
        return BinaryOp(op=p[1], left=p.binary_expression0, right=p.binary_expression1, coord=p.binary_expression0.coord)
    
    @_('unary_expression')
    def binary_expression(self, p):
        return p.unary_expression

    # <unary_expression> ::= <primary_expression>
    #                      | <unary_operator> <unary_expression>
    @_('unary_operator unary_expression')
    def unary_expression(self, p):
        return UnaryOp(op=p.unary_operator, operand=p.unary_expression, coord=p.unary_expression.coord)

    @_('MINUS expression')
    def expression(self, p):
        return UnaryOp('-', p.expression, coord=self._token_coord(p.MINUS))

    @_('primary_expression')
    def unary_expression(self, p):
        return p.primary_expression

    # <unary_operator> ::= "+"
    #                    | "!"
    @_('PLUS')
    def unary_operator(self, p):
        return '+'
    
    @_('MINUS')
    def unary_operator(self, p):
        return '-'

    @_('EXCLAMATION')
    def unary_operator(self, p):
        return '!'

    # <primary_expression> ::= <literal>
    #                        | <location>
    #                        | "<<<" <expression> ">>>"
    #                        | "(" <expression> ")"
    @_('literal')
    def primary_expression(self, p):
        return p.literal

    @_('location')
    def primary_expression(self, p):
        return p.location

    @_('L_HACK expression R_HACK')
    def primary_expression(self, p):
        return PrintStatement(expression=p.expression, coord=self._token_coord(p))

    @_('LPAREN expression RPAREN')
    def primary_expression(self, p):
        return p.expression

    @_('INT_VAL')
    def literal(self, p):
        return Literal(type='int', value=p.INT_VAL, coord=self._token_coord(p))
    
    @_('FLOAT_VAL')
    def literal(self, p):
        return Literal(type='float', value=p.FLOAT_VAL, coord=self._token_coord(p))
    
    @_('STRING_LIT')
    def literal(self, p):
        return Literal(type='string', value=p.STRING_LIT, coord=self._token_coord(p))
    
    @_('TRUE')
    def literal(self, p):
        return Literal(type='int', value='1', coord=self._token_coord(p))

    @_('FALSE')
    def literal(self, p):
        return Literal(type='int', value='0', coord=self._token_coord(p))

    # <location> ::= <identifier>
    @_('identifier')
    def location(self, p):
        return Location(name=p.identifier.name, coord=p.identifier.coord)

    # <identifier> ::= ID
    @_('ID')
    def identifier(self, p):
        return ID(name=p.ID, coord=Coord(p.lineno, self.lexer.find_column(p)))

Complete o código abaixo, escrevendo um método `visit_XXX` para cada tipo de nó AST.

In [15]:
class Visitor(NodeVisitor):
    '''
    Program visitor class. This class uses the visitor pattern. You need to define methods
    of the form visit_NodeName() for each kind of AST node that you want to process.
    '''
    def __init__(self):
        # Keep a reference to current symbol table
        self.symtab = None

        # Add built-in type names (int, float, string)
        self.typemap = {
            "int": IntType,
            "float": FloatType,
            "string": StringType,
        }
        
    def _assert_semantic(self, condition, msg_code, coord, name="", ltype="", rtype=""):
        """Check condition, if false print selected error message and exit"""
        error_msgs = {
             1: f"'{name}' is not defined",
             2: f"Cannot assign type '{rtype}' to type '{ltype}'",
             3: f"Binary operator '{name}' does not have matching LHS/RHS types",
             4: f"Binary operator '{name}' is not supported by type '{ltype}'",
             5: "Break/Continue statement must be inside a loop",
             6: f"The condition expression must be of type 'int', not type '{ltype}'",
             7: "Expression is not of basic type",
             8: f"Right-side operand is not a variable",
             9: f"Name '{name}' is already defined in this scope",
            10: f"Unary operator '{name}' is not supported by type '{ltype}'",
        }
        if not condition:
            msg = error_msgs.get(msg_code)
            print("SemanticError: %s %s" % (msg, coord), file=sys.stdout)
            sys.exit(1)
            
    def visit_BinaryOp(self, node):
        # Visit the left and right expression
        self.visit(node.left)
        ltype = node.left.attrs['uchuck_type']
        self.visit(node.right)
        rtype = node.right.attrs['uchuck_type']
        # TODO:
        # - Make sure left and right operands have the same type
        # - Make sure the operation is supported
        # - Assign the result type to current node
        self._assert_semantic(ltype == rtype, 3, node.coord, name=node.op)
        supported_ops = ltype.binary_ops.union(ltype.rel_ops)
        self._assert_semantic(node.op in supported_ops, 4, node.coord, name=node.op, ltype=ltype.typename)

        if node.op in ltype.rel_ops:
            node.attrs['uchuck_type'] = IntType
        else:
            node.attrs['uchuck_type'] = ltype

    def visit_BreakStatement(self, node):
        loops = self.symtab.lookup('loops')
        self._assert_semantic(len(loops) > 0, 5, node.coord)
        node.attrs['loop'] = loops[-1]

    def visit_ChuckOp(self, node):
        # Visit the expression (on the left)
        self.visit(node.left)
        expr_type = node.left.attrs['uchuck_type']

        # Visit the location (on the right)
        self.visit(node.right)
        loc_type = node.right.attrs['uchuck_type']

        # Make sure the location is a variable, otherwise, return an error (code 8)
        loc = node.right
        is_assignable = loc.__class__.__name__ in ('Location', 'VarDecl')
        self._assert_semantic(is_assignable, 8, coord=getattr(loc, 'coord', node.coord))

        # Check that the assignment is allowed otherwise return a type error (code 2)
        self._assert_semantic(loc_type == expr_type, 2, node.coord, ltype=str(loc_type), rtype=str(expr_type))

        # Assign the type of the location to current node
        node.attrs['uchuck_type'] = loc_type

    def visit_ContinueStatement(self, node):
        loops = self.symtab.lookup('loops')
        self._assert_semantic(len(loops) > 0, 5, node.coord)
        node.attrs['loop'] = loops[-1]

    def visit_ExpressionAsStatement(self, node):
        if node.expression:
            self.visit(node.expression)

    def visit_ExprList(self, node):
        for expr in node.exprs:
            self.visit(expr)
        if node.exprs:
            node.attrs['uchuck_type'] = node.exprs[-1].attrs['uchuck_type']

    def visit_IfStatement(self, node):
        self.visit(node.test)
        test_type = node.test.attrs['uchuck_type']
        self._assert_semantic(test_type == IntType, 6, node.coord, ltype=str(test_type))
        self.visit(node.consequence)
        if node.alternative:
            self.visit(node.alternative)

    def visit_Literal(self, node):
        type_name = node.type
        if type_name == 'bool':
            type_name = 'int'
        node.attrs['uchuck_type'] = self.typemap.get(type_name)

    def visit_Location(self, node):
        decl_node = self.symtab.lookup(node.name)
        self._assert_semantic(decl_node is not None, 1, node.coord, name=node.name)
        node.attrs['defn'] = decl_node
        node.attrs['uchuck_type'] = decl_node.attrs['uchuck_type']

    def visit_PrintStatement(self, node):
        self.visit(node.expression)
        expr_type = node.expression.attrs.get('uchuck_type')
        self._assert_semantic(expr_type is not None, 7, node.coord)
        node.attrs['uchuck_type'] = expr_type

    def visit_Program(self, node):
        # Create a symbol table and assign it to current node
        node.attrs['symtab'] = SymbolTable()
        # Set the reference to current symbol table to the new symbol table
        self.symtab = node.attrs['symtab']
        # Create an empty list to bind loop nodes to nested break/continue statements and insert it into the symbol table
        self.symtab.add('loops', [])
        # Visit all the statements
        for stmt in node.stmts:
            self.visit(stmt)

    def visit_StmtList(self, node):
        if node.stmts:
            for stmt in node.stmts:
                self.visit(stmt)

    def visit_Type(self, node):
        self._assert_semantic(node.name in self.typemap, 1, node.coord, name=node.name)
        node.attrs['uchuck_type'] = self.typemap[node.name]

    def visit_UnaryOp(self, node):
        self.visit(node.operand)
        operand_type = node.operand.attrs['uchuck_type']
        self._assert_semantic(node.op in operand_type.unary_ops, 10, node.coord, name=node.op, ltype=str(operand_type))
        node.attrs['uchuck_type'] = operand_type

    def visit_VarDecl(self, node):
        var_name = node.name.name
        self._assert_semantic(self.symtab.lookup(var_name) is None, 9, node.coord, name=var_name)
        
        self.visit(node.dtype)
        var_type = node.dtype.attrs['uchuck_type']
        
        node.attrs['uchuck_type'] = var_type
        self.symtab.add(var_name, node)

    def visit_WhileStatement(self, node):
        loops = self.symtab.lookup('loops')
        loops.append(node)
        
        self.visit(node.test)
        test_type = node.test.attrs['uchuck_type']
        self._assert_semantic(test_type == IntType, 6, node.coord, ltype=str(test_type))
        
        self.visit(node.body)
        loops.pop()

In [16]:
def print_error(msg, x, y):
    # use stdout to match with the output in the .out test files
    print("Lexical error: %s at %d:%d" % (msg, x, y), file=sys.stdout)

def main(args):
    parser = UChuckParser(print_error)
    with open(args[0], 'r') if len(args) > 0 else sys.stdin as f:
        ast = parser.parse(f.read())
        if ast is not None:
            sema = Visitor()
            sema.visit(ast)
            ast.show(showcoord=True)

## Teste
Para o desenvolvimento inicial, tente executar o analisador semântico em um arquivo de entrada de exemplo, como:

```
/* print values of factorials */
1 => int n;
1 => int value;

while( n < 10 )
{
	value * n => value;
	<<< value >>>;
	n + 1 => n;
}
```

In [17]:
%%file test.uck
/* print values of factorials */
1 => int n;
1 => int value;

while( n < 10 )
{
	value * n => value;
	<<< value >>>;
	n + 1 => n;
}

Overwriting test.uck


E o resultado será semelhante ao texto mostrado abaixo.

```
Program:
    ExpressionAsStatement:
        ChuckOp: @ 2:1
            VarDecl: ID(name=n)
                Type: int @ 2:6
            Literal: int, 1 @ 2:1
    ExpressionAsStatement:
        ChuckOp: @ 3:1
            VarDecl: ID(name=value)
                Type: int @ 3:6
            Literal: int, 1 @ 3:1
    WhileStatement: @ 5:1
        BinaryOp: < @ 5:8
            Location: n @ 5:8
            Literal: int, 10 @ 5:12
        StmtList: @ 6:1
            ExpressionAsStatement:
                ChuckOp: @ 7:2
                    Location: value @ 7:15
                    BinaryOp: * @ 7:2
                        Location: value @ 7:2
                        Location: n @ 7:10
            ExpressionAsStatement:
                PrintStatement: @ 8:2
                    Location: value @ 8:6
            ExpressionAsStatement:
                ChuckOp: @ 9:2
                    Location: n @ 9:11
                    BinaryOp: + @ 9:2
                        Location: n @ 9:2
                        Literal: int, 1 @ 9:6
```

In [18]:
main(["test.uck"])

Program:
    ExpressionAsStatement:
        ChuckOp: @ 2:1
            VarDecl: ID(name=n)
                Type: int @ 2:6
            Literal: int, 1 @ 2:1
    ExpressionAsStatement:
        ChuckOp: @ 3:1
            VarDecl: ID(name=value)
                Type: int @ 3:6
            Literal: int, 1 @ 3:1
    WhileStatement: @ 5:1
        BinaryOp: < @ 5:8
            Location: n @ 5:8
            Literal: int, 10 @ 5:12
        StmtList: @ 6:1
            ExpressionAsStatement:
                ChuckOp: @ 7:2
                    Location: value @ 7:15
                    BinaryOp: * @ 7:2
                        Location: value @ 7:2
                        Location: n @ 7:10
            ExpressionAsStatement:
                PrintStatement: @ 8:2
                    Location: value @ 8:6
            ExpressionAsStatement:
                ChuckOp: @ 9:2
                    Location: n @ 9:11
                    BinaryOp: + @ 9:2
                        Location: n @ 9:2
             

## Anexo

A lista abaixo define os campos e os atributos que devem ser armazenados em cada tipo de nó AST:

```
###########################################################
## Nós do uChuck ##########################################
###########################################################

#   <name>*     - um nó filho
#   <name>**    - uma sequência de nós filhos
#   <name>      - um atributo

# 'defn*'       - um nó que declara um local (decl)
# 'loop*'       - um nó que declara um laço de repetição (while)
# 'symtab'      - tabela de símbolos de um dado escopo
# 'uchuck_type' - um objeto uChuckType

BinaryOp: [op, left*, right*], attrs={'uchuck_type'}

BreakStatement: [], attrs={'loop*'}

ChuckOp: [location*, expression*], attrs={'uchuck_type'}

ContinueStatement: [], attrs={'loop*'}

ExpressionAsStatement: [expression*], attrs={}

ExprList: [exprs**], attrs={'uchuck_type'}

ID: [name], attrs={}

IfStatement: [test*, consequence*, alternative*], attrs={}

Literal: [type, value], attrs={'uchuck_type'}

Location: [name], attrs={'defn*', 'uchuck_type'}

PrintStatement: [expression*], attrs={'uchuck_type'}

Program: [stmts**], attrs={'symtab'}

StmtList: [stmts**], attrs={}

Type: [name], attrs={'uchuck_type'}

UnaryOp: [op, operand*], attrs={'uchuck_type'}

VarDecl: [dtype*, name], attrs={'uchuck_type'}

WhileStatement: [test*, body*], attrs={}
```